In [1]:
from dotenv import load_dotenv
import datetime
from langchain_openai import ChatOpenAI
# Get the current date
current_date = datetime.datetime.now().date()
load_dotenv()
# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10ba73020>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10ba94350>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [2]:
response = chat.invoke("Hello there")
response.content

'Hello! How can I assist you today?'

In [3]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
""" 
from langchain.prompts import ChatPromptTemplate
# Create a ChatPromptTemplate object
template = ChatPromptTemplate.from_template(template_string)

In [4]:
template.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))

In [5]:
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
customer_message = template.format_messages(
    style=customer_style,
    text=customer_email
)
customer_message

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]

In [7]:
customer_response = chat.invoke(customer_message)
customer_response

AIMessage(content="I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 89, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-98cd8d71-10bc-41ce-a285-797052650ef9-0')

In [8]:
style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""
response_pirate = chat.invoke(template.format_messages(
    style=style_pirate,
    text=customer_email
))
response_pirate.content

"Ahoy, me hearties! I be quite vexed that me blender lid flew off and splattered me kitchen walls with smoothie! And to add insult to injury, the warranty don't be coverin' the cost of cleanin' up me kitchen. I be needin' yer help straightaway, me hearty!"

In [9]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [10]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""


In [11]:
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [12]:
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)
response

AIMessage(content='{\n  "gift": true,\n  "delivery_days": 2,\n  "price_value": "It\'s slightly more expensive than the other leaf blowers out there"\n}', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 218, 'total_tokens': 254}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-152cce15-2b78-4176-aa80-ea1494346dc9-0')

In [13]:
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


In [14]:
type(response.content)

str

In [15]:
# we will get error because this is not a json object, it's a string
response.content.gift

AttributeError: 'str' object has no attribute 'gift'

In [16]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [17]:
gift_schema = ResponseSchema(
    name='gift',
    description='Was the item purchased as a gift for someone else? \n'
                'Answer True if yes, False if not or unknown.',
)
delivery_days_schema = ResponseSchema(
    name='delivery_days',
    description='How many days did it take for the product to arrive? \n'
                'If this information is not found, output -1.',
);
price_value_schema = ResponseSchema(
    name='price_value',
    description='Extract any sentences about the value or price, \n'
                'and output them as a comma separated Python list.',
);

response_schema = [ gift_schema, delivery_days_schema, price_value_schema ]

In [18]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased as a gift for someone else? \nAnswer True if yes, False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days did it take for the product to arrive? \nIf this information is not found, output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any sentences about the value or price, \nand output them as a comma separated Python list.', type='string')])

In [19]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift for someone else? \nAnswer True if yes, False if not or unknown.\n\t"delivery_days": string  // How many days did it take for the product to arrive? \nIf this information is not found, output -1.\n\t"price_value": string  // Extract any sentences about the value or price, \nand output them as a comma separated Python list.\n}\n```'

In [20]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(review_template_2)
messages = prompt.format_messages(
    text=customer_review, 
    format_instructions=format_instructions)
messages

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the

In [ ]:
print(messages[0].content)

In [ ]:
response = chat.invoke(messages)
response

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

In [ ]:
output_dict.get('gift')

In [ ]:
output_dict['gift']

In [ ]:
output_dict['delivery_days']